In [3]:
import sys  
sys.path.insert(1, '/home/sanjay/Research/MCTS/Codes/pda_simulator/notebooks')
# sys.path.insert(1, '/mnt/d/PowerTAC/TCS/mcts_thread/pda_simulator/notebooks')

import math
import numpy as np
import pandas as pd

from ipynb.fs.full.config import Config

import warnings
warnings.filterwarnings('ignore')

In [4]:
config = Config()

In [33]:
class Observer:

    def __init__(self):
        self.current_timeslot = 0
        self.hour_ahead = 0
        self.initial_needed_enery_mcts_broker = 0
        self.needed_enery_per_broker = self.initial_needed_enery_mcts_broker

        self.moving_avg_err_bal = 0.0
        self.mcts_simulation = 5000
        self.HOUR_AHEAD_AUCTIONS = 24
        self.total_hours_ahead = 0
        self.arr_clearing_prices = 0
        self.balacing_price = 90.0

    def set_time(self, current_timeslot, hour_ahead):
        self.current_timeslot = current_timeslot
        self.hour_ahead = hour_ahead

In [ ]:
class Action:
    
    def __init__(self, index=0): 
        self.index = index
        self.quantityRatio = np.array([0.0, 0.16, 0.33, 0.5, 0.66, 0.84, 1.0])
        self.multiplier = np.array([[1.0, 0.0], [0.84, 0.16], [0.66, 0.33], [0.50, 0.50], [0.33, 0.66], [0.16, 0.84], [0.0, 1.0]])
        self.max_action = self.quantityRatio.shape[0] * self.multiplier.shape[0]

    def get_action_index(self):
        return self.index
    
    def get_action_size(self):
        return self.max_action
    
    def get_qauntity_action_size(self):
        return self.quantityRatio.shape[0]
    
    def get_limit_price(self, minPrice, maxPrice, index):
      factor = self.multiplier[index]
      return minPrice*factor[0] + maxPrice*factor[1]
    
    def get_quantity_to_bid(self, index, neededKwh):
      return self.quantityRatio[index] * neededKwh

    def get_limit_price_fractions(self, index):
      quotient = index / self.quantityRatio.shape[0]
      return self.multiplier[quotient]

    def get_quantity_ratio(self, index):
      reminder = index % self.quantityRatio.shape[0]
      return self.quantityRatio[reminder]
    
    def get_wholesale_bid(self, minPrice, maxPrice, neededKwh):
      quotient = self.index / self.quantityRatio.shape[0]
      reminder = self.index % self.quantityRatio.shape[0]

      price = self.get_limit_price(minPrice, maxPrice, quotient)
      quantity = self.get_quantity_to_bid(reminder, neededKwh)
      return price, quantity

    def set_action_index(self, index):
        self.index = index

    def to_string(self):
      return str(self.index)

In [ ]:
class TreeNode:

    def __init__(self, node=None):
        self.epsilon = 1e-6
        self.min_mwh = 0.001
        self.fixed_vol = 50.0

        self.children = dict()

        if node == None:
            self.n_visits = 0
            self.tot_value = 0
            self.current_node_cost = 0

            self.hour_ahead_auction = 0
            self.applied_action = 0
            self.limit_price_fractions = 0
            self.quantity_ratio = 0
        else:
            self.n_visits = node.n_visits
            self.tot_value = node.tot_value
            self.current_node_cost = node.current_node_cost

            self.hour_ahead_auction = node.hour_ahead_auction
            self.applied_action = node.applied_action
            self.limit_price_fractions = node.limit_price_fractions 
            self.quantity_ratio = node.quantity_ratio

    
    def unvisited_children(self, tn):
        count = 0

        for child in tn.children.items():
            if child.n_visits == 0:
                count += 1

        return count
    

    public void runMonteCarlo(Action action, MCTS mcts, Observer ob, BrokerContext broker, VVGenericMDP baselineMDP, AskGenerator askGenerator) throws InterruptedException      // Traversal till depth 0
    {
        Double neededEnergy = Double.valueOf(ob.neededEneryPerBroker);
        Double iniNeededEnergy = Double.valueOf(ob.initialNeededEneryMCTSBroker);
        List<TreeNode> visited = new LinkedList<>();
        TreeNode cur = this;

        visited.add(this);

        if (cur.isLeaf())
        {
        System.out.println("cur.isLeaf() is 0! This should not happen!");
        }

        Double avgMCP = 0.0;
        int countClearing = 0;

        while (!cur.isLeaf() && (neededEnergy != 0.0))
        {
        if (cur.children == null)
        {
            cur.expand(mcts);
        }

        int unvisitedChildren = unvisitedChildren(cur);

        if (unvisitedChildren == (action.getActionSize()))              // If this is first visit select random child
        {
            cur = cur.selectRandom(mcts);
        }
        else
        {
            double balancingPrice = (avgMCP == 0.0) ? 150.0 : 3*avgMCP;      // Testing
            cur = cur.select(mcts, ob, balancingPrice);     // UCT based selection
        }
        visited.add(cur);

        double[] retValue = simulation(cur, ob, neededEnergy, iniNeededEnergy, broker, baselineMDP, askGenerator);
        neededEnergy = Double.valueOf(neededEnergy.doubleValue() - retValue[0]);

        if(retValue[1] != 0.0)
        {
            avgMCP = (avgMCP*countClearing + retValue[1])/(countClearing+1);
            countClearing++;
        }
        }

        double bprice = (avgMCP == 0.0) ? 150.0 : (3*avgMCP);         //  3 times the avg clearing price     // Testing

        double balancingSimCost = -Math.abs(neededEnergy.doubleValue()) * bprice;

        Collections.reverse(visited);

        Iterator<TreeNode> iter = visited.iterator();

        double simCost = balancingSimCost;

        while (iter.hasNext())
        {
        TreeNode node = iter.next();
        simCost += node.currentNodeCost;
        node.updateStats(simCost);
        }

        // for(TreeNode item : visited)
        // {
        //   System.out.println(item.toString());
        // }
    }

    def select_random(self, mcts):
        i = np.random.randint(0, mcts.action.get_action_size())
        return self.children.get(i)
    

    def expand(self, mcts):
        n_actions = mcts.action.get_action_size();
        self.children = dict();
        new_hour_ahead_auction = self.hourAheadAuction - 1;

        for i in range(n_actions):
            if (i == 0) or (i % mcts.action.get_quantity_action_size() != 0):
                tn = TreeNode()
                tn.hour_ahead_auction = new_hour_ahead_auction
                tn.applied_action = i
                tn.limit_price_fractions = mcts.action.get_limit_price_fractions(i)
                tn.quantity_ratio = mcts.action.get_quantity_ratio(i)
                self.children.update({i: tn})
            else:
                self.children.update({i: self.children.get(0)})


    def final_select(self, ob):
        selected = None
        best_value = -1e9;

        for child in self.children:
            total_point = child.total_value

            if total_point > best_value:
                selected = child
                best_value = total_point

        return selected;


    def select(self, mcts, ob, balancing_price):
        selected = None
        best_value = -1e9
    
        for child in self.children:
            n_visit_value = 0
            total_point = 0

            if child.n_visits == 0:
                n_visit_value = 1 + epsilon
                total_point = child.tot_value
            else:
                n_visit_value = child.n_visits + epsilon
                total_point = child.tot_value;
    
            dividend = -balancing_price * abs(ob.initial_needed_enery_mcts_broker)
            total_point = 1 - total_point / dividend

            visit_point = sqrt(2 * log(self.n_visits + 1) / n_visit_value)
            rand_point = np.random.random() * epsilon;
            uct_value = total_point + visit_point + rand_point;

            if uct_value > best_value:
                selected = child
                best_value = uct_value

        return selected;


    def simulation(TreeNode tn, Observer ob, Double neededMWh, Double iniNeededEnergy, BrokerContext broker, VVGenericMDP baselineMDP, AskGenerator askGenerator)
    {
        Random r = new Random();

        List<OrderWrapper> bids = new ArrayList<>();   // store all the bids here (generate self and opponent bids)
        Integer auctionProximity = tn.hourAheadAuction+1;

        // System.out.println("Proximity: " + auctionProximity);

        // Prepare Own Bid
        baselineMDP.setState(auctionProximity);
        // baselineMDP.getActions().printProbableLimitPrices(baselineMDP.getState());

        Pair<Double, Double> limitPriceRange = baselineMDP.getActions().getLimitPriceRange(baselineMDP.getState(), true);

        // System.out.println(auctionProximity + " :: " + tn.limitPriceFractions[0] + " : " + limitPriceRange.getKey() + " :: " + tn.limitPriceFractions[1] + " : " +
        // limitPriceRange.getValue() + " :: " + tn.quantityRatio + " : " + neededMWh);

        Double limitPrice = tn.limitPriceFractions[0]*limitPriceRange.getKey() + tn.limitPriceFractions[1]*limitPriceRange.getValue();
        Double quantity = tn.quantityRatio*neededMWh;

        if (quantity > minMWh)
        {
        OrderWrapper ownBid = new OrderWrapper(new Order(broker.getBroker().getUsername(), quantity, -Math.abs(limitPrice)));
        bids.add(ownBid);
        }

        // Prepare Opponent1's bid
        // try
        // {
        //   Double neededMWhForOpponent = r.nextDouble()*fixedVol;

        //   if (neededMWhForOpponent > minMWh)
        //   {
        //     OrderWrapper opponentBid = new OrderWrapper(OpponentBidGenerator.prepareOrder("ZI", auctionProximity,neededMWhForOpponent, neededMWhForOpponent));
        //     bids.add(opponentBid);
        //   }
        // }
        // catch(Exception e) {}

        // Prepare Opponent2's bid
        // try
        // {
        //   Double neededMWhForOpponent = r.nextDouble()*fixedVol;

        //   if (neededMWhForOpponent > minMWh)
        //   {
        //     OrderWrapper opponentBid = new OrderWrapper(OpponentBidGenerator.prepareOrder("Linear", auctionProximity,neededMWhForOpponent, neededMWhForOpponent));
        //     bids.add(opponentBid);
        //   }
        // }
        // catch(Exception e) {}

        // GenCOs' Asks
        List<List<OrderWrapper>> allAsks = askGenerator.sampleAsks(auctionProximity);
        List<OrderWrapper> asks = allAsks.get(r.nextInt(allAsks.size()));

        List<OrderWrapper> clonedAsks = new ArrayList<OrderWrapper>(asks.size());
        for (OrderWrapper order : asks) {
            clonedAsks.add(new OrderWrapper(order));
        }

        // System.out.println(bids);
        // System.out.println(clonedAsks);

        Double[] cleared = AuctionService.clearAuction(bids, clonedAsks);   // clearingPrice and clearingQuantity
        // System.out.println(cleared[0] + " :: " + cleared[1]);

        double currentCost = cleared[1]*cleared[0];
        double totalBidVolume = cleared[1];

        tn.currentNodeCost = -currentCost;

        return new double[] { totalBidVolume, cleared[0] };
    }


    def is_leaf(self):
        return self.hourAheadAuction == 0       # No more chances
    

    def update_stats(self, sim_cost):
        self.tot_value = (self.tot_value * self.n_visits + sim_cost) / (self.n_visits + 1)
        self.n_visits += 1

    
    public String toString()
  {
    String ret = "Tree Node Data\n";

    ret += "Number of Visits: " + this.nVisits + "\n";
    ret += "Total Value: " + this.totValue + "\n";
    ret += "Proximity: " + this.hourAheadAuction + "\n";
    ret += "Applied Action Index: " + this.appliedAction + "\n";
    // ret += "Limitprice Fractions: " + this.limitPriceFractions[0] + "," + this.limitPriceFractions[1] + "\n";
    // ret += "Quantity Ratio: " + this.quantityRatio + "\n";

    /*if(this.hourAheadAuction == 2)
    {
      if(this.children != null)
      {
        for(TreeNode child : this.children)
        {
          ret += child.toString() + "\n";
        }
      }
      else
        ret += "null\n";
    }*/

    return ret;
  }


In [ ]:
class MCTS_Desc:

    def __init__(self, total_demand, number_of_bids=1, buy_limit_price_min=-100.0, buy_limit_price_max=-1.0, sell_limit_price_min=0.5, sell_limit_price_max=100.0, id='ZIP'):
        self.seed_value = 0 # seed value will be overidden
        self.id = id
        self.total_demand = total_demand 
        self.cleared_demand = 0
        self.min_bid_quant = 0.01
        self.number_of_bids = number_of_bids
        self.buy_limit_price_min = buy_limit_price_min
        self.buy_limit_price_max = buy_limit_price_max
        self.sell_limit_price_min = sell_limit_price_min
        self.sell_limit_price_max = sell_limit_price_max
        self.last_mcp = 40.0

        self.bid_price_by_future_timeslot = dict()
        self.ask_price_by_future_timeslot = dict()

    def set_cleared_demand(self, cleared_demand):
        self.cleared_demand += cleared_demand
        
    def set_last_mcp(self, mcp):
        self.last_mcp = mcp

    def bids(self, timeslot, currentTimeslot):

        rem_quantity = self.total_demand - self.cleared_demand
        
        if rem_quantity < self.min_bid_quant:
            return None
        
        amount_needed = rem_quantity
        remaining_tries = timeslot - currentTimeslot
        
        bids = list()
        if remaining_tries > 0:
            
            if amount_needed > 0.0:
                
                if timeslot not in self.bid_price_by_future_timeslot.keys():
                    lpd = LimitPriceDeterminants(np.random.random()*self.buy_limit_price_min, -0.1 , currentTimeslot)
                    self.bid_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                else:
                    lpd = self.bid_price_by_future_timeslot.get(timeslot)
                    sdelta = lpd.learningRate * (-self.last_mcp - lpd.limitPrice)         # For bid, limitPrice is negetive, so "-lastMCP"
                    lpd.executionTimeslot = currentTimeslot
                    lpd.delta = lpd.momentumCoefficient * lpd.delta + (1 - lpd.momentumCoefficient) * sdelta
                    lpd.profitMargin = max(-1.0, lpd.profitMargin + (lpd.delta/lpd.limitPrice))
                    lpd.limitPrice = max(self.buy_limit_price_min, lpd.limitPrice * (1 + lpd.profitMargin))
                    self.bid_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                
            else:
                
                if timeslot not in self.ask_price_by_future_timeslot.keys():
                    lpd = LimitPriceDeterminants((config.sell_limit_price_max + config.sell_limit_price_min)/2, 0.1, currentTimeslot)
                    self.ask_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice
                else:
                    lpd = self.ask_price_by_future_timeslot.get(timeslot)
                    sdelta = lpd.learningRate * (self.last_mcp - lpd.limitPrice) 
                    lpd.executionTimeslot = currentTimeslot
                    lpd.delta = lpd.momentumCoefficient * lpd.delta + (1 - lpd.momentumCoefficient) * sdelta
                    lpd.profitMargin = max(-1.0, lpd.profitMargin + (lpd.delta/lpd.limitPrice))
                    lpd.limitPrice = max(self.sell_limit_price_max, lpd.limitPrice * (1 + lpd.profitMargin))
                    self.ask_price_by_future_timeslot.update({timeslot: lpd})
                    bid = lpd.limitPrice

            bids.append([self.id, bid, amount_needed])
        else:
            bids.append([self.id, -config.market_order_bid_price, amount_needed])

        return bids